# Impact of Climate Change on Maize Agricultural Yield in Kenya


Mir Tahmid

mirtahmid@gmail.com

# Dataset Overview:

The dataset consists of 43 entries and 27 columns, including various agricultural, weather, and environmental factors from Kenya between 1981 and 2023. Key columns include:

area_harvested_usda_1000ha, production_usda_1000ha, yield_usda_1000ha: Agricultural data.

dew_temp_C, soil_temp_L1_C, irradiation_J_m2, precipitation_era5_mm: Weather and environmental factors.

year: The time component for modeling.

# What we are going to do in the notebook:

Step by Step Procedures

**Import required libraries**

**Loading and Glimpse of dataset**

**Perform data preprocessing and feature engineering.**

**Conduct EDA with bar charts, pie charts, and other visualizations.**

**Build ARIMA, SARIMAX, and Prophet models.**

**Compare their performance using RMSE, MSE, and MAPE.**

**Visualizing the Forecasts**

**Prophet Forecast Components**

In [1]:
!pip install pandas numpy seaborn matplotlib statsmodels scikit-learn prophet skimpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.7 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.3.0
    Uninstalling typeguard-4.3.0:
      Successfully uninstalled typeguard-4.3.0
  Attempting uninstall: polars
    Found existing installation: polars 1.7.1
    Uninstalling polars-1.7.1:
      Successfully uninstalled polars-1.7.1
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 5.5.6
    Uninstalling ipykernel-5.5.6:
      Successfully uninstalled ipykernel-5.5.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipykernel==5.5.6, but you have ipykernel 6.29.5 which is incompatible.


# Import Required Libraries

In [2]:
# Import necessary libraries for data manipulation, visualization, and modeling
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Statistical models
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA

# Machine learning metrics
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

# Prophet for forecasting
from prophet import Prophet

# Skimpy for data overview
import skimpy as sk

# For calculating square root
from math import sqrt

# Ignore warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

# Data Loading and Glimpse of the Dataset

In [3]:
# Load the detrended dataset
detrended_data = pd.read_csv('/kaggle/input/kenya-1981-2023-detrended-var/Kenya_1981_2023_detrended_var.csv')

# Display the first few rows to understand the structure
print("First 5 rows of the dataset:")
display(detrended_data.head())

# Display dataset information
print("\nDataset Information:")
detrended_data.info()

# Quick summary statistics using skimpy
print("\nSummary Statistics:")
skim_summary = sk.skim(detrended_data)
display(skim_summary)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/kenya-1981-2023-detrended-var/Kenya_1981_2023_detrended_var.csv'

#  Data Preprocessing and Feature Engineering

In [ ]:
# Select relevant columns: 'year', 'yield_usda_1000ha', and 'production_usda_1000ha'
data_cleaned = detrended_data[['year', 'yield_usda_1000ha', 'production_usda_1000ha']].copy()

# Set 'year' as the index
data_cleaned.set_index('year', inplace=True)

# Check for missing values
missing_values = data_cleaned.isnull().sum()

# Convert the Series to DataFrame for styling
missing_values_df = missing_values.to_frame(name='Missing Values')

# Display missing values with color styling
print("Missing Values:")
missing_values_styled = missing_values_df.style.applymap(lambda x: 'color: lime' if x == 0 else 'color: green')
display(missing_values_styled)

# Since the data is already detrended, no further detrending is required

# Display the cleaned data with color styling
print("\nCleaned Data:")
cleaned_data_styled = data_cleaned.style.background_gradient(cmap='YlGn')
display(cleaned_data_styled)

# Conduct Exploratory Data Analysis (EDA)

**Plotting Yield USDA (1000ha) over Time**

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(data_cleaned['yield_usda_1000ha'], label='Yield USDA (1000ha)', color='lime')
plt.title('Detrended Yield USDA (1000ha) over Time (1981-2023)')
plt.xlabel('Year')
plt.ylabel('Yield USDA (1000ha)')
plt.legend()
plt.show()

# Plotting Production USDA (1000ha) over Time

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(data_cleaned['production_usda_1000ha'], label='Production USDA (1000ha)', color='green')
plt.title('USDA Production (1000ha) over Time (1981-2023)')
plt.xlabel('Year')
plt.ylabel('Production USDA (1000ha)')
plt.legend()
plt.show()

# Comparing Yield and Production on the Same Plot

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(data_cleaned['yield_usda_1000ha'], label='Yield USDA (1000ha)', color='lime')
plt.plot(data_cleaned['production_usda_1000ha'], label='Production USDA (1000ha)', color='green')
plt.title('Yield vs Production USDA (1000ha) (1981-2023)')
plt.xlabel('Year')
plt.ylabel('USDA (1000ha)')
plt.legend()
plt.show()

# Distribution of USDA Production (1000ha)

In [ ]:
plt.figure(figsize=(8, 6))

# Create a histogram with KDE
sns.histplot(data_cleaned['production_usda_1000ha'], bins=20, kde=True, color='green', alpha=0.6)

# Overlay the KDE line with lime color
sns.kdeplot(data_cleaned['production_usda_1000ha'], color='lime', linewidth=2)

# Title and labels
plt.title('Distribution of USDA Production (1000ha)')
plt.xlabel('Production USDA (1000ha)')
plt.ylabel('Frequency')

# Show plot
plt.show()

# Correlation Analysis

In [ ]:
# Select only numeric columns
numeric_data = detrended_data.select_dtypes(include=['float64', 'int64'])

# Calculate the correlation matrix for numeric variables
correlation_matrix = numeric_data.corr()

# Plot the heatmap using a lime-green color palette (YlGn)
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='YlGn', linewidths=0.5, vmin=-1, vmax=1)
plt.title('Correlation Matrix Heatmap for All Numeric Variables (Lime Color Scheme)')
plt.show()

# Correlation Between Yield and Production

In [ ]:
# Calculate the correlation matrix for yield and production
correlation = data_cleaned[['yield_usda_1000ha', 'production_usda_1000ha']].corr()

# Print the correlation matrix
print("Correlation between Yield and Production:")

# Style the correlation DataFrame with lime and green colors
correlation_styled = correlation.style.background_gradient(cmap='YlGn')
display(correlation_styled)

# Building ARIMA, SARIMAX, and Prophet Models

**Splitting the Data into Training and Testing Sets**

In [ ]:
# Define the target variable
target = 'yield_usda_1000ha'

# Split the data: Last 10 years for testing, the rest for training
train = data_cleaned[target].iloc[:-10]
test = data_cleaned[target].iloc[-10:]

print(f"Training Data Shape: {train.shape}")
print(f"Testing Data Shape: {test.shape}")

# Building the ARIMA Model

In [ ]:
# Define and fit the ARIMA model
# You may need to tune the order (p,d,q) based on ACF and PACF plots
arima_order = (5, 0, 1)  # Example order; adjust as needed
arima_model = ARIMA(train, order=arima_order)
arima_fit = arima_model.fit()

# Summary of the ARIMA model
print("ARIMA Model Summary:")
display(arima_fit.summary())

# Make predictions
arima_pred = arima_fit.forecast(steps=len(test))
print("\nARIMA Predictions:")
display(arima_pred)

# Building the SARIMAX Model

In [ ]:
# Define and fit the SARIMAX model
sarimax_order = (1, 0, 1)
seasonal_order = (1, 1, 0, 12)  # Example seasonal order; adjust as needed
sarimax_model = SARIMAX(train, order=sarimax_order, seasonal_order=seasonal_order)
sarimax_fit = sarimax_model.fit(disp=False)

# Summary of the SARIMAX model
print("SARIMAX Model Summary:")
display(sarimax_fit.summary())

# Make predictions
sarimax_pred = sarimax_fit.forecast(steps=len(test))
print("\nSARIMAX Predictions:")
display(sarimax_pred)

# Building the Prophet Model

In [ ]:
# Prepare data for Prophet
prophet_data = data_cleaned.reset_index().rename(columns={'year': 'ds', 'yield_usda_1000ha': 'y'})

# Split into training and testing for Prophet
prophet_train = prophet_data.iloc[:-10]
prophet_test = prophet_data.iloc[-10:]

# Initialize and fit the Prophet model
from prophet import Prophet
prophet_model = Prophet()
prophet_model.fit(prophet_train)

# Create a dataframe to hold predictions
future = prophet_model.make_future_dataframe(periods=10, freq='Y')
prophet_forecast = prophet_model.predict(future)

# Extract the forecasted values
prophet_pred = prophet_forecast['yhat'][-10:].values

# Create a DataFrame to display the predictions with lime and green colors
prophet_pred_df = pd.DataFrame({
    'Year': prophet_test['ds'],
    'Actual': prophet_test['y'],
    'Prophet Prediction': prophet_pred
})

# Style the DataFrame using lime-green color scheme
prophet_pred_styled = prophet_pred_df.style.background_gradient(cmap='YlGn')

print("\nProphet Predictions with Lime and Green:")
display(prophet_pred_styled)

# Comparing Model Performance using RMSE, MSE, and MAPE

In [ ]:
# Calculate performance metrics for ARIMA
arima_rmse = sqrt(mean_squared_error(test, arima_pred))
arima_mse = mean_squared_error(test, arima_pred)
arima_mape = mean_absolute_percentage_error(test, arima_pred)

# Calculate performance metrics for SARIMAX
sarimax_rmse = sqrt(mean_squared_error(test, sarimax_pred))
sarimax_mse = mean_squared_error(test, sarimax_pred)
sarimax_mape = mean_absolute_percentage_error(test, sarimax_pred)

# Calculate performance metrics for Prophet
prophet_rmse = sqrt(mean_squared_error(test, prophet_pred))
prophet_mse = mean_squared_error(test, prophet_pred)
prophet_mape = mean_absolute_percentage_error(test, prophet_pred)

# Create a comparison DataFrame
comparison_df = pd.DataFrame({
    'Model': ['ARIMA', 'SARIMAX', 'Prophet'],
    'RMSE': [arima_rmse, sarimax_rmse, prophet_rmse],
    'MSE': [arima_mse, sarimax_mse, prophet_mse],
    'MAPE': [arima_mape, sarimax_mape, prophet_mape]
})

# Display the DataFrame with lime and green gradient
comparison_df_styled = comparison_df.style.background_gradient(cmap='YlGn')

print("Model Performance Comparison:")
display(comparison_df_styled)

# Visualizing the Forecasts

**Plotting Actual vs Predicted Values**

In [ ]:
plt.figure(figsize=(14, 7))

# Plot training data in blue
plt.plot(train.index, train, label='Training Data', color='lightgreen')

# Plot actual data in green (Actual Yield)
plt.plot(test.index, test, label='Actual Yield', color='green')

# ARIMA predictions in red
plt.plot(test.index, arima_pred, label='ARIMA Predictions', color='red')

# SARIMAX predictions in lime (custom color)
plt.plot(test.index, sarimax_pred, label='SARIMAX Predictions', color='lime')

# Prophet predictions in orange
plt.plot(test.index, prophet_pred, label='Prophet Predictions', color='orange')

# Title and labels
plt.title('Actual vs Predicted Yield USDA (1000ha)')
plt.xlabel('Year')
plt.ylabel('Yield USDA (1000ha)')

# Display legend
plt.legend()

# Show plot
plt.show()

# Prophet Forecast Components

In [ ]:
# Plot Prophet forecast components
components = prophet_model.plot_components(prophet_forecast)

# Customize trend, seasonality, and holidays plots
for ax in components.get_axes():
    ax.lines[0].set_color('lime')  # Main line to lime color
    ax.fill_between(ax.get_lines()[0].get_xdata(), ax.get_lines()[0].get_ydata(), color='green', alpha=0.3)  # Fill to green

# Display the customized plots
plt.show()
